In [ ]:
prompt = """
  You are a friendly leave request copilot.your job is tocapture the following details from the user and give them as a Jason file.
  
  extract the following information:
  1.Leave type (these are the only leave types available. Annual, Casual, Lieu Leave, Maternity, Medical)
  2.Starting date (in the format YYYY-MM-DD)
  3.Ending date (in the format YYYY-MM-DD)
  4.No Days (How many days take for leave can be calculated 'ending date-starting date')
  5.Full day (This is for full day leave)
  6.HalfDay (This is for half day leave)
  7.Reason for leave (brief explanation)
  8.Description (optional, for additional details)
  9. generate remark using above data as following example
  For a full day leave:'Dear [reporting person],I am writing to request leave from [Starting date] to [Ending date] due to [reason for leave].
  I understand the importance of my responsibilities and will ensure that my work is up to date before my departure. 
  I will also make sure to provide any necessary handovers to my colleagues.Thank you for considering my request. 
  I look forward to your approval.'

  For a half day leave:'Dear [Recipient's Name],I am writing to request half-day leave on [Starting date] due to [reason for leave]. 
  I would like to take leave for [halfday] of the day.I understand the importance of my responsibilities and will ensure that my work is up to date before my departure. 
  Thank you for considering my request. I look forward to your approval.'
  Example JSON Format for Full Day Leave:
  {{  \"Leave Type\": \"Annual Leave\",  \"From\": \"2023-09-06\",  \"To\": \"2023-09-10\",  \"No Days\": 5,  \"FullDay\": \"Full Day\",  \"HalfDay\": None,  \"Leave Reason\": \"Wedding\",  \"Remark\":'Dear Dilshan Kavinda,I am writing to request leave from 2023-09-06 to 2023-09-10 due to a Wedding.I understand the importance of my responsibilities and will ensure that my work is up to date before my departure. Thank you for considering my request. I look forward to your approval.'}} 
  Example JSON Format for Half Day Leave:
  {{  \"Leave Type\": \"Casual Leave\",  \"From\": \"2023-09-06\",  \"To\": \"2023-09-06\",  \"No Days\": 0.5,  \"FullDay\": None,  \"HalfDay\": \"1st Half\",  \"Leave Reason\": \"Family Need\",  \"Remark\":'Dear Dilshan Kavinda,I am writing to request half-day leave on 2023-09-06 due to personal reasons. I would like to take leave for first half of the day.I understand the importance of my responsibilities and will ensure that my work is up to date before my departure. Thank you for considering my request. I look forward to your approval.'}}

  If any information is missing or unclear, prompt the user again for specific details
  If any information is missing or unclear,just ask to provide the missing information asking questions one by one.
  If the dates are invalid or the reason is too vague, ask for clarification.
  VERY IMPORTANT: Make sure you Do not make up or guess ANY extra information. 
  Only extract what exactly is in the user request.
  You respond in a short, very conversational friendly style.

  once you create the jason ask you to save the leave request.To do that ask user to type 'SAVE'."""

In [1]:
import pandas as pd
import json


import pandas as pd
import json

# Provided string with JSON data
input_string = '''Alright, here's the JSON file and remark:

{
  "Leave Type": "Annual ",
  "From": "2024-02-03",
  "To": "2024-02-03",
  "No Days": 1,
  "FullDay": "Full Day",
  "HalfDay": "None",
  "Leave Reason": "Feeling unwell",
  "Remark": "Dear Dilshan Kavinda,\\n\\nI am writing to request sick leave on 2024-02-03 due to feeling unwell.\\n\\nI understand the importance of my responsibilities and will ensure that my work is up to date before my departure. I will also make sure to provide any necessary handovers to my colleagues.\\n\\nThank you for considering my request. I look forward to your approval."
}

Please review the JSON file and remark. If everything looks good, type 'SAVE' to save the leave request.'''

# Extract JSON data from the string
json_start = input_string.find('{')
json_end = input_string.rfind('}')
json_data = input_string[json_start:json_end + 1]

# Load JSON data into a Python dictionary
leave_data = json.loads(json_data)

# Additional data
leave_data['Leave ID'] = 123  # Replace with an appropriate leave ID
leave_data['Employee ID'] = 456  # Replace with an appropriate employee ID

# Map leave types to numerical values
leave_type_mapping = {
    "annual leave": 1,
    "casual leave": 2,
    "lieu leave": 3,
    "maternity leave": 4,
    "medical leave": 5
}

leave_data['Leave Type'] = leave_type_mapping.get(leave_data['Leave Type'].lower().strip(), leave_data['Leave Type'])

# Create a Pandas DataFrame
df = pd.DataFrame([leave_data])




In [2]:
df

,Leave Type,From,To,No Days,FullDay,HalfDay,Leave Reason,Remark,Leave ID,Employee ID
0,Annual,2024-02-03,2024-02-03,1,Full Day,None,Feeling unwell,"Dear Dilshan Kavinda,\n\nI am writing to reque...",123,456


In [3]:
leave_data = pd.read_csv('leave_data.csv')
leave_allocation = pd.read_csv("Leave_Allocation.csv")

In [4]:
employee_id = 1
filtered_df = leave_data[leave_data['Employee ID'] == employee_id]
merged_df = pd.merge(leave_allocation,filtered_df, left_on='LeaveTypeCode', right_on='Leave Type',how='left')
result_df = merged_df.groupby('LeaveTypeName')['No Days'].sum().reset_index()
final_df = pd.merge(result_df, leave_allocation, left_on='LeaveTypeName', right_on='LeaveTypeName')
new_final_df = final_df[['LeaveTypeName','No Days','AllocatedNoOfDays']]

In [5]:
new_final_df

,LeaveTypeName,No Days,AllocatedNoOfDays
0,ANNUAL,0.0,10
1,CASUAL,1.0,10
2,Lieu Leave,0.0,10
3,MATERNITY,1.0,10
4,MEDICAL,0.0,10


In [6]:
leave_data = pd.read_csv('leave_data.csv')
leave_allocation = pd.read_csv("Leave_Allocation.csv")

def leave_insights(employee_id,leave_data,leave_allocation):
    filtered_df = leave_data[leave_data['Employee ID'] == employee_id]
    merged_df = pd.merge(leave_allocation,filtered_df, left_on='LeaveTypeCode', right_on='Leave Type',how='left')
    result_df = merged_df.groupby('LeaveTypeName')['No Days'].sum().reset_index()
    final_df = pd.merge(result_df, leave_allocation, left_on='LeaveTypeName', right_on='LeaveTypeName')
    new_final_df = final_df[['LeaveTypeName','No Days','AllocatedNoOfDays']]
    return new_final_df

insights = leave_insights(employee_id,leave_data,leave_allocation)

text_insights = '**LEAVE BALANCE:**\n\n'

for index, row in insights.iterrows():
    leave_type = row['LeaveTypeName']
    actual_days = row['No Days']
    allocated_days = row['AllocatedNoOfDays']

    if actual_days <= allocated_days:
        # insight = f"For {leave_type} leave, you took {actual_days} days out of {allocated_days} allocated days. No overuse."
        insight =f"- **{leave_type}:** You can have ***{allocated_days-actual_days} leaves.*** ({actual_days} days out of {allocated_days} allocated days were used.)\n"
    else:
        overuse = actual_days - allocated_days
        insight = f"For {leave_type} leave, you exceeded the allocated days by {overuse} days. Please review."

    text_insights = text_insights+'\n'+insight


In [7]:
text_insights

'**LEAVE BALANCE:**\n\n\n- **ANNUAL:** You can have ***10.0 leaves.*** (0.0 days out of 10 allocated days were used.)\n\n- **CASUAL:** You can have ***9.0 leaves.*** (1.0 days out of 10 allocated days were used.)\n\n- **Lieu Leave:** You can have ***10.0 leaves.*** (0.0 days out of 10 allocated days were used.)\n\n- **MATERNITY:** You can have ***9.0 leaves.*** (1.0 days out of 10 allocated days were used.)\n\n- **MEDICAL:** You can have ***10.0 leaves.*** (0.0 days out of 10 allocated days were used.)\n'